In [1]:
import torch
from supervoice_hybrid import UnitTextTokenizer

In [2]:
tokenizer = UnitTextTokenizer()

Using the cached tokenizer of nar_t2u_aligner. Set `force` to `True` to download again.


In [7]:
print(tokenizer.encode("get! 667?"))
print(tokenizer.encode("гэt! 667?"))

tensor([ 42,  58,  43,  47, 237,  42, 475, 475, 480, 128])
tensor([ 42, 189, 470,  47, 237,  42, 475, 475, 480, 128])
